In [0]:
!pip install dlib

In [0]:
# import required packages
import cv2
import dlib
import time
from io import BytesIO
import requests
import numpy as np
from urllib.request import urlopen

ModuleNotFoundError: ignored

In [0]:
#input_image = './image.jpeg'
input_dir = './face_detection.json'
input_weight = 'mmod_human_face_detector.dat'
# initialize hog + svm based face detector
hog_face_detector = dlib.get_frontal_face_detector()
# initialize cnn based face detector with the weights
cnn_face_detector = dlib.cnn_face_detection_model_v1(input_weight)

In [0]:
filePath = '/Users/hiroshi/Desktop/118B_project/face_detection.json'
def parseData(file_path):
    with open(file_path,'r') as f:
        for data in f:
            data = data.replace("null","None")
            yield(eval(data))
INPUT_IMAGE_PATH = list(parseData(input_dir)) # For non-test
#INPUT_IMAGE_PATH = [list(parseData(input_dir))[0]] # For test

In [0]:
def url_to_image(url):
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

In [0]:
def HOG_predict(image):
    #start = time.time()

    # apply face detection (hog)
    faces_hog = hog_face_detector(image, 1)

    #end = time.time()
    #print("Execution Time (in seconds) :")
    #print("HOG : ", format(end - start, '.2f'))

    pred_HOG = []
    # loop over detected faces
    for face in faces_hog:
        x_min = face.left()
        y_max = face.top()
        x_max = face.right()
        y_min = face.bottom()
        pred_HOG.append([x_min, y_min, x_max, y_max])
    return pred_HOG

In [0]:
def CNN_predict(image):
    #start = time.time()

    # apply face detection (cnn)
    faces_cnn = cnn_face_detector(image, 1)

    #end = time.time()
    #print("CNN : ", format(end - start, '.2f'))

    pred_CNN = []
    # loop over detected faces
    for face in faces_cnn:
        x_min = face.rect.left()
        y_max = face.rect.top()
        x_max = face.rect.right()
        y_min = face.rect.bottom()
        pred_CNN.append([x_min, y_min, x_max, y_max])
    
    return pred_CNN

In [0]:
## Main function
HOG_pred_list = []
CNN_pred_list = []
for image_path in INPUT_IMAGE_PATH:
    image = url_to_image(image_path['content'])
    #if image is None:
    #    print("Could not read input image")
    #    exit()
    HOG_pred_list.append(HOG_predict(image))
    CNN_pred_list.append(CNN_predict(image))

In [0]:
with open('./HOG_pred.txt','w') as f:
    for item in HOG_pred_list:
        f.write("%s\n" % item)
with open('./CNN_pred.txt','w') as f:
    for item in CNN_pred_list:
        f.write("%s\n" % item)